# Recherche Azure AI avec l'intégration de NVIDIA NIM et LlamaIndex

Dans ce notebook, nous allons montrer comment exploiter les modèles d'IA de NVIDIA et LlamaIndex pour créer un pipeline puissant de génération augmentée par la recherche (RAG). Nous utiliserons les LLMs et les embeddings de NVIDIA, les intégrerons avec Azure AI Search comme magasin vectoriel, et effectuerons du RAG pour améliorer la qualité et l'efficacité de la recherche.

## Avantages
- **Évolutivité** : Exploitez les modèles de langage de grande taille de NVIDIA et Azure AI Search pour une recherche évolutive et efficace.
- **Efficacité des coûts** : Optimisez la recherche et la récupération avec un stockage vectoriel efficace et des techniques de recherche hybride.
- **Haute performance** : Combinez des LLMs puissants avec une recherche vectorisée pour des réponses plus rapides et précises.
- **Qualité** : Maintenez une qualité de recherche élevée en ancrant les réponses des LLMs avec des documents pertinents récupérés.

## Prérequis
- 🐍 Python 3.9 ou supérieur
- 🔗 [Service Azure AI Search](https://learn.microsoft.com/azure/search/)
- 🔗 Clé API NVIDIA pour accéder aux LLMs et Embeddings de NVIDIA via les microservices NVIDIA NIM

## Fonctionnalités couvertes
- ✅ Intégration des LLMs de NVIDIA (nous utiliserons [Phi-3.5-MOE](https://build.nvidia.com/microsoft/phi-3_5-moe))
- ✅ Embeddings de NVIDIA (nous utiliserons [nv-embedqa-e5-v5](https://build.nvidia.com/nvidia/nv-embedqa-e5-v5))
- ✅ Modes avancés de récupération avec Azure AI Search
- ✅ Indexation de documents avec LlamaIndex
- ✅ RAG utilisant Azure AI Search et LlamaIndex avec les LLMs de NVIDIA

C'est parti !


In [ ]:
!pip install azure-search-documents==11.5.1
!pip install --upgrade llama-index
!pip install --upgrade llama-index-core
!pip install --upgrade llama-index-readers-file
!pip install --upgrade llama-index-llms-nvidia
!pip install --upgrade llama-index-embeddings-nvidia
!pip install --upgrade llama-index-postprocessor-nvidia-rerank
!pip install --upgrade llama-index-vector-stores-azureaisearch
!pip install python-dotenv

## Installation et exigences
Créez un environnement Python en utilisant une version de Python >3.10.

## Premiers pas !


Pour commencer, vous avez besoin d'une `NVIDIA_API_KEY` pour utiliser les modèles NVIDIA AI Foundation :  
1) Créez un compte gratuit avec [NVIDIA](https://build.nvidia.com/explore/discover).  
2) Cliquez sur le modèle de votre choix.  
3) Sous Input, sélectionnez l'onglet Python, puis cliquez sur **Get API Key** et ensuite sur **Generate Key**.  
4) Copiez et enregistrez la clé générée sous NVIDIA_API_KEY. À partir de là, vous devriez avoir accès aux points de terminaison.  


In [3]:
import getpass
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
    assert nvidia_api_key.startswith("nvapi-"), f"{nvidia_api_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvidia_api_key


## Exemple RAG utilisant LLM et Embedding
### 1) Initialiser le LLM
`llama-index-llms-nvidia`, également connu sous le nom de connecteur LLM de NVIDIA, vous permet de vous connecter à des modèles compatibles et de générer à partir de ceux disponibles dans le catalogue API de NVIDIA. Consultez ici une liste des modèles de complétion de chat : https://build.nvidia.com/search?term=Text-to-Text

Ici, nous utiliserons **mixtral-8x7b-instruct-v0.1**


In [75]:
from llama_index.core import Settings
from llama_index.llms.nvidia import NVIDIA

# Here we are using mixtral-8x7b-instruct-v0.1 model from API Catalog
Settings.llm = NVIDIA(model="microsoft/phi-3.5-moe-instruct", api_key=os.getenv("NVIDIA_API_KEY"))

### 2) Initialiser l'Embedding
`llama-index-embeddings-nvidia`, également connu sous le nom de connecteur d'Embeddings de NVIDIA, vous permet de vous connecter à des modèles compatibles et de générer à partir de ceux disponibles dans le catalogue API de NVIDIA. Nous avons sélectionné `nvidia/nv-embedqa-e5-v5` comme modèle d'Embedding. Consultez ici une liste de modèles d'Embedding de texte : https://build.nvidia.com/nim?filters=usecase%3Ausecase_text_to_embedding%2Cusecase%3Ausecase_image_to_embedding


In [6]:
from llama_index.embeddings.nvidia import NVIDIAEmbedding

Settings.embed_model = NVIDIAEmbedding(model="nvidia/nv-embedqa-e5-v5", api_key=os.getenv("NVIDIA_API_KEY"))

### 3) Créer un magasin de vecteurs Azure AI Search


In [76]:
import logging
import sys
import os
import getpass
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from IPython.display import Markdown, display
from llama_index.vector_stores.azureaisearch import AzureAISearchVectorStore, IndexManagement


search_service_api_key = os.getenv('AZURE_SEARCH_ADMIN_KEY') or getpass.getpass('Enter your Azure Search API key: ')
search_service_endpoint = os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT') or getpass.getpass('Enter your Azure Search service endpoint: ')
search_service_api_version = "2024-07-01"
credential = AzureKeyCredential(search_service_api_key)

# Index name to use
index_name = "llamaindex-nvidia-azureaisearch-demo"

# Use index client to demonstrate creating an index
index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=credential,
)

# Use search client to demonstrate using existing index
search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=credential,
)

In [ ]:
vector_store = AzureAISearchVectorStore(
    search_or_index_client=index_client,
    index_name=index_name,
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
    id_field_key="id",
    chunk_field_key="chunk",
    embedding_field_key="embedding",
    embedding_dimensionality=1024, # dimensionality for nv-embedqa-e5-v5 model
    metadata_string_field_key="metadata",
    doc_id_field_key="doc_id",
    language_analyzer="en.lucene",
    vector_algorithm_type="exhaustiveKnn",
    # compression_type="binary" # Option to use "scalar" or "binary". NOTE: compression is only supported for HNSW
)

In [20]:
from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.core.text_splitter import TokenTextSplitter

# Configure text splitter (nv-embedqa-e5-v5 model has a limit of 512 tokens per input size)
text_splitter = TokenTextSplitter(separator=" ", chunk_size=500, chunk_overlap=10)

# Load documents
documents = SimpleDirectoryReader(
    input_files=["data/txt/state_of_the_union.txt"]
).load_data()
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create index with text splitter
index = VectorStoreIndex.from_documents(
    documents,
    transformations=[text_splitter],
    storage_context=storage_context,
)

### 5) Créer un moteur de requête pour poser des questions sur vos données

Voici une requête utilisant une recherche vectorielle pure dans Azure AI Search et ancrant la réponse à notre LLM (Phi-3.5-MOE)


In [69]:
query_engine = index.as_query_engine()
response = query_engine.query("Who did the speaker mention as being present in the chamber?")
display(Markdown(f"{response}"))

 The speaker mentioned the Ukrainian Ambassador to the United States, along with other members of Congress, the Cabinet, and various officials such as the Vice President, the First Lady, and the Second Gentleman, as being present in the chamber.

Voici une requête utilisant la recherche hybride dans Azure AI Search.


In [70]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from IPython.display import Markdown, display
from llama_index.core.schema import MetadataMode

# Initialize hybrid retriever and query engine
hybrid_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.HYBRID)
hybrid_query_engine = RetrieverQueryEngine(retriever=hybrid_retriever)

# Query execution
query = "What were the exact economic consequences mentioned in relation to Russia's stock market?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))

 The Russian stock market experienced a significant drop, losing 40% of its value. Additionally, trading had to be suspended due to the ongoing situation.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

building a coalition of other freedom-loving nations from Europe and the Americas to Asia and Africa to confront Putin. 

I spent countless hours unifying our European allies. We shared with the world in advance what we knew Putin was planning and precisely how he would try to falsely justify his aggression.  

We countered Russia’s lies with truth.   

And now that he has acted the free world is holding him accountable. 

Along with twenty-seven members of the European Union including France, Germany, Italy, as well as countries like the United Kingdom, Canada, Japan, Korea, Australia, New Zealand, and many others, even Switzerland. 

We are inflicting pain on Russia and supporting the people of Ukraine. Putin is now isolated from the world more than ever. 

Together with our allies –we are right now enforcing powerful economic sanctions. 

We are cutting off Russia’s largest banks from the international financial system.  



#### Analyse de la recherche vectorielle
La réponse du LLM capture avec précision les principales conséquences économiques mentionnées dans le texte source concernant le marché boursier russe. Plus précisément, elle indique que le marché boursier russe a subi une chute importante, perdant 40 % de sa valeur, et que les échanges ont été suspendus en raison de la situation en cours. Cette réponse correspond bien aux informations fournies dans le texte source, ce qui montre que le LLM a correctement identifié et résumé les détails pertinents concernant l'impact sur le marché boursier résultant des actions de la Russie et des sanctions imposées.

#### Commentaire sur les nœuds sources
Les nœuds sources fournissent un compte rendu détaillé des conséquences économiques auxquelles la Russie a été confrontée en raison des sanctions internationales. Le texte souligne que le marché boursier russe a perdu 40 % de sa valeur et que les échanges ont été suspendus. En outre, il mentionne d'autres répercussions économiques, telles que la dévaluation du rouble et l'isolement plus large de l'économie russe. La réponse du LLM a efficacement extrait les points essentiels de ces nœuds, en se concentrant sur l'impact sur le marché boursier comme demandé dans la requête.


Maintenant, examinons une requête où la recherche hybride ne fournit pas une réponse bien fondée :


In [71]:
# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it does mention that the events discussed are happening in the current era and that the actions taken are in response to Putin's aggression. For the precise date, one would need to refer to external sources or historical records.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

our forces are not engaged and will not engage in conflict with Russian forces in Ukraine.  

Our forces are not going to Europe to fight in Ukraine, but to defend our NATO Allies – in the event that Putin decides to keep moving west.  

For that purpose we’ve mobilized American ground forces, air squadrons, and ship deployments to protect NATO countries including Poland, Romania, Latvia, Lithuania, and Estonia. 

As I have made crystal clear the United States and our Allies will defend every inch of territory of NATO countries with the full force of our collective power.  

And we remain clear-eyed. The Ukrainians are fighting back with pure courage. But the next few days weeks, months, will be hard on them.  

Putin has unleashed violence and chaos.  But while he may make gains on the battlefield – he will pay a continuing high price over the long run. 

And a proud Ukrainian people, who have known 30 years  of independence,

### Recherche Hybride : Analyse de la Réponse du LLM
La réponse du LLM dans l'exemple de Recherche Hybride indique que le contexte fourni ne précise pas la date exacte de l'invasion de l'Ukraine par la Russie. Cette réponse suggère que le LLM utilise les informations disponibles dans les documents sources tout en reconnaissant l'absence de détails précis dans le texte.

La réponse est correcte en identifiant que le contexte mentionne des événements liés à l'agression de la Russie, mais ne précise pas la date exacte de l'invasion. Cela démontre la capacité du LLM à comprendre les informations fournies tout en repérant les lacunes dans le contenu. Le LLM incite efficacement l'utilisateur à consulter des sources externes ou des archives historiques pour obtenir la date exacte, affichant ainsi une certaine prudence lorsque les informations sont incomplètes.

### Analyse des Nœuds Sources
Les nœuds sources dans l'exemple de Recherche Hybride contiennent des extraits d'un discours abordant la réponse des États-Unis face aux actions de la Russie en Ukraine. Ces nœuds mettent en avant l'impact géopolitique global et les mesures prises par les États-Unis et leurs alliés en réaction à l'invasion, mais ils ne mentionnent pas la date exacte de l'invasion. Cela correspond à la réponse du LLM, qui identifie correctement que le contexte ne contient pas l'information précise sur la date.


In [72]:
# Initialize hybrid retriever and query engine
semantic_reranker_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.SEMANTIC_HYBRID)
semantic_reranker_query_engine = RetrieverQueryEngine(retriever=semantic_reranker_retriever)

# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = semantic_reranker_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it mentions that the event occurred six days before the speech was given. To determine the precise date, one would need to know the date of the speech.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. 

### Hybrid w/Reranking : Analyse de la réponse du LLM
Dans l'exemple Hybrid w/Reranking, la réponse du LLM fournit un contexte supplémentaire en précisant que l'événement s'est produit six jours avant le discours. Cela indique que le LLM est capable de déduire la date de l'invasion en se basant sur le moment du discours, même s'il doit encore connaître la date exacte du discours pour être précis.

Cette réponse démontre une capacité améliorée à utiliser des indices contextuels pour fournir une réponse plus informative. Elle met en avant l'avantage du reranking, où le LLM peut accéder à des informations plus pertinentes et les prioriser afin de donner une approximation plus proche du détail recherché (c'est-à-dire la date de l'invasion).

### Analyse des nœuds sources
Les nœuds sources dans cet exemple incluent des références au moment de l'invasion de la Russie, mentionnant spécifiquement qu'elle s'est produite six jours avant le discours. Bien que la date exacte ne soit toujours pas explicitement indiquée, les nœuds fournissent un contexte temporel qui permet au LLM de donner une réponse plus nuancée. L'inclusion de ce détail illustre comment le reranking peut améliorer la capacité du LLM à extraire et déduire des informations à partir du contexte fourni, aboutissant à une réponse plus précise et informative.


**Remarque :**  
Dans ce notebook, nous avons utilisé les microservices NIM de NVIDIA provenant du catalogue d'API NVIDIA.  
Les API mentionnées ci-dessus, `NVIDIA (llms)`, `NVIDIAEmbedding`, et [Azure AI Search Semantic Hybrid Retrieval (classement intégré)](https://learn.microsoft.com/azure/search/semantic-search-overview). Notez que ces API peuvent également prendre en charge des microservices auto-hébergés.  

**Exemple :**  
```python
NVIDIA(model="meta/llama3-8b-instruct", base_url="http://your-nim-host-address:8000/v1")```



---

**Avertissement** :  
Ce document a été traduit à l'aide du service de traduction automatique [Co-op Translator](https://github.com/Azure/co-op-translator). Bien que nous nous efforcions d'assurer l'exactitude, veuillez noter que les traductions automatisées peuvent contenir des erreurs ou des inexactitudes. Le document original dans sa langue d'origine doit être considéré comme la source faisant autorité. Pour des informations critiques, il est recommandé de faire appel à une traduction professionnelle humaine. Nous déclinons toute responsabilité en cas de malentendus ou d'interprétations erronées résultant de l'utilisation de cette traduction.
